In [1]:
import numpy as np
from pathlib import Path
import sys

In [13]:
class TreeNode:
    def __init__(self, level, V, W, taken):
        self.level = level # niveau dans l’arbre de recherche >=0
        self.V = V # valeur de la solution courante
        self.W = W # poids de la solution courante
        self.taken = taken # liste des index d’objets de la solution courante
    def __str__(self):
        return str((self.level, self.V, self.W, self.taken))

In [14]:
class TreeNode:
    """Classe pour les noeuds de l'arbre de recherche. Chaque noeud est caractérisé par : 
    - son niveau dans l'arbre de recherche (>= 0)
    - la valeur de la solution courante
    - le poids de la solution courante
    - la liste des index d'objets de la solution courante
    """

    def __init__(self, level, V, W, taken):
        self.level = level  # niveau dans l'arbre de recherche >= 0
        self.V = V          # valeur de la solution courante
        self.W = W          # poids de la solution courante
        self.taken = taken  # liste des index d'objets de la solution courante

    def __str__(self):
        return str((self.level, self.V, self.W, self.taken)) # affichage d'un noeud
    

class KPBB:
    """Classe pour le problème du sac à dos avec l'algorithme de branch and bound.
    - cap : capacité du sac à dos
    - values : liste des valeurs des objets
    - weights : liste des poids des objets 
    - unordered : liste des objets avec leur index
    - ordered : liste des objets triés par rapport à leur ratio valeur/poids
    """
    def __init__(self, cap, values, weights): # Initialisation des données
        self.cap = cap
        self.values = values
        self.weights = weights
        self.unordered = [((v,w), i) for i, (v,w) in enumerate(zip(self.values, self.weights))] # liste des objets avec leur index
        self.ordered = sorted([((v,w),i) for i, (v,w) in enumerate(zip(self.values, self.weights))], key=lambda tup: tup[0][0]/tup[0][1], reverse=True) # liste des objets triés par rapport à leur ratio valeur/poids.


    # calcul de la solution initiale
    def initial_solution(self):
        taken = []
        V = 0
        W = 0
        for (v, w), i in self.ordered:
            if W + w <= self.cap:
                taken.append(i)
                V += v
                W += w
        return V, W, taken


    # fonction d'évaluation d'un noeud
    def nodeEvaluation(self, index, node):
        V, W = node.V, node.W
        for i in range(index, len(self.ordered)):
            (v, w), j = self.ordered[i]
            if W + w <= self.cap:
                V += v
                W += w
            else:
                V += (self.cap - W) * v / w
                break
        return V


    def solve(self):
        best = TreeNode(-1, 0, 0, []) 
        racine = TreeNode(0, 0, 0, []) 
        queue = [racine] 
        
        while queue: 
            node = queue.pop() 
            if node.level == len(self.ordered): 
                if node.V > best.V: 
                    best = node  
            else:
                i = node.level
                (v, w), j = self.ordered[i]
                if node.W + w <= self.cap:
                    taken = node.taken + [j]
                    V = node.V + v
                    W = node.W + w
                    enfant = TreeNode(i+1, V, W, taken)
                    if enfant.V > best.V:
                        queue.append(enfant)
                if self.nodeEvaluation(i+1, node) > best.V:
                    queue.append(TreeNode(i+1, node.V, node.W, node.taken))
        return best.V, best.W, best.taken

In [15]:

cap = 10 
values = [40, 50, 100, 95, 30] 
poids = [2, 3, 1, 5, 3]
kpbb = KPBB(cap, values, poids)
V, W, taken = kpbb.solve()
print("Valeur optimale :", V)
print("Poids optimal :", W) 
print("Objets pris :", taken)
print("Valeurs des objets pris :", [values[i] for i in taken])


Valeur optimale : 245
Poids optimal : 9
Objets pris : [2, 3, 1]
Valeurs des objets pris : [100, 95, 50]


In [28]:
def execute_all(directory):
    p=Path('./instances_01_KP/'+directory)
    print('Fichier        Solution     Optimum')
    for f in p.iterdir():
        first_row=np.loadtxt(f.absolute(),max_rows=1)
        fich=np.loadtxt(f.absolute(),max_rows=int(first_row[0])+1)
        clf=KPBB(fich[0][1],fich[1:,0],fich[1:,1])
        V, W, taken = clf.solve()
        print(f'{f.name}-> {taken} -> {V}')

In [ ]:
execute_all('low-dimensional')

Fichier        Solution     Optimum
f2_l-d_kp_20_878-> [6, 0, 7, 13, 15] -> 186.0
f8_l-d_kp_23_10000-> [19, 20, 22] -> 2777.0
f4_l-d_kp_4_11-> [1, 3] -> 23.0
f10_l-d_kp_20_879-> [7, 19, 6, 9, 16] -> 186.0
f5_l-d_kp_15_375-> [4, 13, 3, 8, 1] -> 198.950163
f3_l-d_kp_4_20-> [3, 2] -> 28.0
f1_l-d_kp_10_269-> [8, 7, 0, 3] -> 206.0
f6_l-d_kp_10_60-> [1, 0] -> 38.0
f7_l-d_kp_7_50-> [0, 3] -> 107.0
f9_l-d_kp_5_80-> [3, 2, 1, 4] -> 109.0


In [30]:
execute_all('large_scale')

Fichier        Solution     Optimum
knapPI_3_100_1000_1-> [45, 71] -> 1197.0
knapPI_3_2000_1000_1-> [1465, 1714] -> 2198.0
knapPI_3_500_1000_1-> [61, 431] -> 2195.0
knapPI_3_200_1000_1-> [159, 3] -> 1197.0


KeyboardInterrupt: 